# Obtenção da repetição origem-destino com grids de 100m, agora com todas as áreas da parte rural de Ann Arbor

In [ ]:
import skmob
from skmob.utils import utils, constants
from skmob.tessellation import tilers
from skmob.utils.plot import plot_gdf

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
import folium
from folium.plugins import HeatMap
import matplotlib as mpl
import matplotlib.pyplot as plt

## 1) Filtrar pelo id do veículo e concatenar em um único DataFrame

In [ ]:
import pandas as pd
import glob
import numpy as np
files = glob.glob("/media/joao/Seagate Expansion Drive 09032021/22122022/ved_descompactados/*.csv")

df = pd.DataFrame()
for f in files:
    plt = pd.read_csv(f, sep = ",")
    df = pd.concat([plt,df], ignore_index=False)
    df = df.loc[(df['VehId'] == 630)]

df

## 2) Ordernar pelas colunas do trip_id pelo Timestamp

In [ ]:
df = df.sort_values(by=['Trip','Timestamp(ms)'])


df

## 1.1) Analisar todos os arquivos de trajetórias

In [ ]:
df['Trip'].nunique()

In [ ]:
#df['VehId'].nunique()

In [ ]:
df['Trip'].unique()

In [ ]:
#df['VehId'].unique()

## Obs.: antes de prosseguir, determino o datetime

In [ ]:
#df = df.sort_values(by='DayNum')

df['datetime'] = pd.to_datetime(df['DayNum'], unit='D', origin=pd.Timestamp('2017-11-01'))


print(df['datetime'].min())

print(df['datetime'].max())


print(df['DayNum'].min())

print(df['DayNum'].max())

print(df.shape)

In [ ]:
t = pd.DataFrame()

t = pd.date_range(start='2018-05-05 16:46:47.212838400', end='2018-05-05 16:46:47.212838400', periods=424)


df['datetime'] = t

df.datetime

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime']).round('s')

df.datetime

## 3) Criação da malha de grids por 100m a hipotenusa

In [ ]:
import pandas as pd
import geopandas as gpd

#vei8 = df

In [ ]:
from skmob.tessellation import tilers
from skmob.preprocessing import filtering

In [ ]:
#import pandas as pd
import skmob
#import geopandas as gpd
from skmob import preprocessing

In [ ]:
tessellation = gpd.read_file('limite_geral_tess.gpkg')

tessellation = tessellation.to_crs('EPSG:4326')

tessellation.plot()

In [ ]:
tessellation

In [ ]:
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=gpd.points_from_xy(df["Longitude[deg]"], df["Latitude[deg]"]))

gdf

In [ ]:
from skmob.tessellation import tilers

tessellation = tilers.tiler.get("squared", 
                                base_shape=tessellation, 
                                meters=100)

tessellation

In [ ]:
# tessellation.to_file('tessellation100.gpkg', driver='GPKG')

In [ ]:
#tdf.columns

In [ ]:
#fdf = tdf.to_flowdataframe(tessellation=tessellation, self_loops=False)

#print(fdf.head())

In [ ]:
mtdf = gdf.sjoin(tessellation, how='inner', predicate='within')

mtdf

## 4) Posições iniciais e finais para cada Trip

In [ ]:
import movingpandas as mpd

#### Obs.: se houver apenas 1 Trip, usar Timestamp(ms) no lugar do datetime.

In [ ]:
traj = mpd.TrajectoryCollection(mtdf, traj_id_col='Trip', obj_id_col='VehId', t='datetime', x='lng', y='lat', crs='epsg:4326')
traj

In [ ]:
# traj.plot(column='speed', legend=True, figsize=(9,5))

In [ ]:
start = traj.get_start_locations()
end = traj.get_end_locations()

In [ ]:

start['lon'] = start.geometry.x
start['lat'] = start.geometry.y

In [ ]:

end['lon'] = end.geometry.x
end['lat'] = end.geometry.y

In [ ]:
start.plot()

In [ ]:
end.plot()

In [ ]:
start['direção'] = 'Origem'
end['direção'] = 'Destino'



## 5) Tabela com os pontos iniciais, finais e o id da trajetória

In [ ]:
pontos = start.merge(end, on='Trip')

pontos

## 6) Tabela com os grids de origem e de destino

In [ ]:
pontos.columns

In [ ]:
#mtdf = mtdf[['uid', 'direção', 'tile_ID', 'lon', 'lat']]

#mtdf.rename({'uid':'veículo_id', 'tile_ID':'grid'}, axis=1, inplace=True)
pontos = pontos[['VehId_x', 'Trip', 'direção_x','direção_y', 'tile_ID_x','tile_ID_y','lon_x','lat_x','lon_y','lat_y']]

pontos.rename({ 'direção_x':'Origens','direção_y':'Destinos', 'tile_ID_x':'grid_origem','tile_ID_y':'grid_destino'}, axis=1, inplace=True)


## 7) Somatório e análise

#### Primeiro, faço uma exploração por partes:

In [ ]:
A = pd.DataFrame()

A['repetições'] = pontos.groupby(['grid_origem']).size().reset_index().groupby(['grid_origem']).sum()

A.shape

In [ ]:
B = pd.DataFrame()

B['repetições'] = pontos.groupby(['grid_destino']).size().reset_index().groupby(['grid_destino']).sum()

B.shape

In [ ]:
A.reset_index()

In [ ]:
A['repetições'].sum()

In [ ]:
A['repetições'].max()

In [ ]:
B.reset_index()

In [ ]:
B['repetições'].sum()

In [ ]:
B['repetições'].max()

In [ ]:
df = A.merge(B, left_on='grid_origem', right_on='grid_destino')

df

#### Agora, faço o procedimento final:

In [ ]:
df_final=pd.DataFrame()

df_final['repetições'] = pontos.groupby(['grid_origem','grid_destino']).size().reset_index().groupby(['grid_origem','grid_destino']).sum()

df_final.shape

In [ ]:
df_final = df_final.reset_index()

In [ ]:
df_final

In [ ]:
df_final['repetições'].sum()

In [ ]:
df_final['repetições'].nlargest(15)

## FONTE ORIGINAL DOS GRIDS, ANTES DAS TRANSFORMAÇÕES: https://data.humdata.org/dataset/hotosm_usa_michigan_populated_places 